In [47]:
# solutions should be to send bob a smooth prime then pohlig-hellman, but it's screwed up for some reason
# ok I fixed it but for some reason we aren't getting the real bob secret
# alright for 1536 bit prime the discrete log doesn't get the right answer 100% of the time. to check just do pow(g, b, p) and see if B is correct

In [40]:
from Crypto.Util.number import getPrime
from sympy import isprime

def get_smooth_prime(bits, smoothness=16):
    p = 2
    p_factors = [p]
    while p.bit_length() < bits - 2 * smoothness:
        factor = getPrime(smoothness)
        p_factors.append(factor)
        p *= factor
    bitcnt = (bits - p.bit_length()) // 2
    while True:
        prime1 = getPrime(bitcnt)
        prime2 = getPrime(bitcnt)
        tmpp = p * prime1 * prime2
        if tmpp.bit_length() < bits:
            bitcnt += 1
            continue
        if tmpp.bit_length() > bits:
            bitcnt -= 1
            continue
        if isprime(tmpp + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = tmpp + 1
            break
    p_factors.sort()
    return (p, p_factors)
sprime, sprime_factors = get_smooth_prime(1536)

In [41]:
from sympy.utilities.misc import as_int
from sympy.ntheory.residue_ntheory import _discrete_log_trial_mul, _discrete_log_index_calculus, _discrete_log_shanks_steps, _discrete_log_pollard_rho, _discrete_log_pohlig_hellman
from collections import Counter
def smooth_discrete_log(n, a, b, sp_factors: list, order=None, prime_order=None):
    from math import sqrt, log
    n, a, b = as_int(n), as_int(a), as_int(b)
    if order is None:
        factors = dict(Counter(sp_factors).most_common())
        order = 1
        for px, kx in factors.items():
            order *= px**kx
        # Now the `order` is the order of the group and factors = factorint(order)
        # The order of `b` divides the order of the group.
        order_factors = {}
        for p, e in factors.items():
            i = 0
            for _ in range(e):
                if pow(b, order // p, n) == 1:
                   order //= p
                   i += 1
                else:
                    break
            if i < e:
                order_factors[p] = e - i

    if prime_order is None:
        prime_order = isprime(order)

    if order < 1000:
        return _discrete_log_trial_mul(n, a, b, order)
    elif prime_order:
        # Shanks and Pollard rho are O(sqrt(order)) while index calculus is O(exp(2*sqrt(log(n)log(log(n)))))
        # we compare the expected running times to determine the algorithmus which is expected to be faster
        if 4*sqrt(log(n)*log(log(n))) < log(order) - 10:  # the number 10 was determined experimental
            return _discrete_log_index_calculus(n, a, b, order)
        elif order < 1000000000000:
            # Shanks seems typically faster, but uses O(sqrt(order)) memory
            return _discrete_log_shanks_steps(n, a, b, order)
        return _discrete_log_pollard_rho(n, a, b, order)

    return _discrete_log_pohlig_hellman(n, a, b, order, order_factors)
a = pow(2, 65537, sprime)
smooth_discrete_log(sprime, a, 2, sprime_factors)

65537

In [42]:
# only found accepted value of g is 2 so far
# A will be sussed out for [0, 2]. this probably doesn't matter
# p will be sussed out if not 1536 bits
# doesn't check for strong prime, we might be able to discrete log 
from pwnlib.tubes.remote import remote
from cryptolib import json_send, diffie_hellman_decrypt_flag, json_recv
from json import loads, dumps
from sympy import prime
r = remote("socket.cryptohack.org", 13378)
r.recvuntil(b"Intercepted from Alice: ")
alice_public = json_recv(r)
p, g, A = int(alice_public['p'], 16), int(alice_public['g'], 16), int(alice_public['A'], 16)
r.recvuntil(b"Intercepted from Bob: ")
B = int(json_recv(r)['B'], 16)
r.recvuntil(b"Intercepted from Alice: ")
alice_aes = json_recv(r)
iv, encrypted = alice_aes['iv'], alice_aes['encrypted']
r.recvuntil(b"Bob connects to you, send him some parameters: ")


json_send(r, {"p": hex(sprime), "g": hex(2), "A": hex(3)})
r.recvuntil(b"Bob says to you: ")
B_ = int(loads(r.recvline())['B'], 16)

In [43]:
smooth_discrete_log(sprime, B_, 2, sprime_factors)

1919572943691512325783103720167834163677411292709378502535498859989993544026380143919501049584589675317643993465536543895780854808442293000014297210200227069779643763121704810281976733978781152126062646602812482025293137787739116693980988513420732289020477701182639042794562638875881378349771734410919106042203493166198706573467903966100368713572415175654342828296086659529676015616513470105470901979846373335352656586302787870238998914215908919919219987614105175

In [44]:
b = 1919572943691512325783103720167834163677411292709378502535498859989993544026380143919501049584589675317643993465536543895780854808442293000014297210200227069779643763121704810281976733978781152126062646602812482025293137787739116693980988513420732289020477701182639042794562638875881378349771734410919106042203493166198706573467903966100368713572415175654342828296086659529676015616513470105470901979846373335352656586302787870238998914215908919919219987614105175
pow(g, b, p)

1966270916217592359951676292828353789783843380410548453232464354948931346091056847611148699597441303286170947817233031631021092250186143266695446008797995598427726291854816681491432301281698548646454200606211899229521112313338141928795614048789917663164666503304428773520125901611456230575542070557469811329398244290382779816263538635910784392500499733451254540473589815041612184948013568818235656785287486503105364932675297907092338998390607579988135467845797831

In [46]:
b = 1919572943691512325783103720167834163677411292709378502535498859989993544026380143919501049584589675317643993465536543895780854808442293000014297210200227069779643763121704810281976733978781152126062646602812482025293137787739116693980988513420732289020477701182639042794562638875881378349771734410919106042203493166198706573467903966100368713572415175654342828296086659529676015616513470105470901979846373335352656586302787870238998914215908919919219987614105175
secret = pow(A, b, p)
diffie_hellman_decrypt_flag(secret, iv, encrypted)

'crypto{uns4f3_pr1m3_sm4ll_oRd3r}'

In [ ]:
from sympy.ntheory.residue_ntheory import discrete_log
discrete_log(sprime, B_, 2)

In [63]:
from Crypto.Util.number import inverse, getPrime
getPrime(1536)

1654870807489741666166712504001835292191126761314153027652346620421649434971976149797565316194747539944117759592727080300884221376989401231040881816893012817556464231807564159340373964933698451719001929968306564707001329275843374696183366614294595402193412758168750368154847505511192419166115921252541617998733354198358398654112905044050386599728877889736600813889719539656152096140540920070332687041208010905095616591952709825797184093420086492088130092808262803

b'B'

In [4]:
p, g, A

(2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919,
 2,
 1842456548509985448001501955708317543923668882702387731149842197334091656598019912729386918817972088534618034054199283446233163815232643357049841287518070880509964732159087089881097780930507595659799099021074417551186847518515681575923228780734810350836185507503936008727914189692134425269770264193394331644892088233122500403697047904867774929619033647804401486714172117981128849420027222802210971219407887958527960013507993788267466267953204739503158129814352528)

In [5]:
B

714216756598305735603046246561487555788511628802563305910135524591202127744114943901795325807949250885402626819853303127965411915027566384701087862456809063646195165193592925563073929195170506328048913324457052878973166150910160579628348050428532539130082307794315984411600494511723848533327684422477621927785771984975971767068564356745941560374525498015524036641953306600952157983344056291242132311780666645344225431766551347466472617080772291014016878534899147